In [1]:
import tensorflow as tf
import snake_game
import random

pygame 2.1.2 (SDL 2.0.18, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [6]:
env = snake_game.snake_env()
states = 7
actions = env.action_space

In [3]:
def define_states(frame_size,snake_head,snake_tail,food,dir):
    foodq1 = 0
    foodq2 = 0
    foodq3 = 0
    foodq4 = 0
    collisionfront = 0
    collisionleft = 0
    collisionright = 0

    if dir == 'right':
        if food[0] >= snake_head[0] and food[1] <= snake_head[1]:
            foodq1 = 1
        if food[0] <= snake_head[0] and food[1] <= snake_head[1]:
            foodq2 = 1
        if food[0] <= snake_head[0] and food[1] >= snake_head[1]:
            foodq3 = 1
        if food[0] >= snake_head[0] and food[1] >= snake_head[1]:
            foodq4 = 1
        if snake_head[0] + 1 > frame_size[0] or (snake_head[0] + 1, snake_head[1]) in snake_tail:
            collisionfront = 1
        if snake_head[1] - 1 < 0 or (snake_head[0], snake_head[1] - 1) in snake_tail:
            collisionleft = 1
        if snake_head[1] + 1 > frame_size[1] or (snake_head[0], snake_head[1] + 1) in snake_tail:
            collisionright = 1
    elif dir == 'left':
        if food[0] <= snake_head[0] and food[1] >= snake_head[1]:
            foodq1 = 1
        if food[0] >= snake_head[0] and food[1] >= snake_head[1]:
            foodq2 = 1
        if food[0] >= snake_head[0] and food[1] <= snake_head[1]:
            foodq3 = 1
        if food[0] <= snake_head[0] and food[1] <= snake_head[1]:
            foodq4 = 1
        if snake_head[0] - 1 < 0 or (snake_head[0] - 1, snake_head[1]) in snake_tail:
            collisionfront = 1
        if snake_head[1] + 1 > frame_size[1] or (snake_head[0], snake_head[1] + 1) in snake_tail:
            collisionleft = 1
        if snake_head[1] - 1 < 0 or (snake_head[0], snake_head[1] - 1) in snake_tail:
            collisionright = 1
    elif dir == 'up':
        if food[1] <= snake_head[1] and food[0] <= snake_head[0]:
            foodq1 = 1
        if food[1] <= snake_head[1] and food[0] >= snake_head[0]:
            foodq2 = 1
        if food[1] >= snake_head[1] and food[0] >= snake_head[0]:
            foodq3 = 1
        if food[1] >= snake_head[1] and food[0] <= snake_head[0]:
            foodq4 = 1
        if snake_head[1] - 1 < 0 or (snake_head[0], snake_head[1] - 1) in snake_tail:
            collisionfront = 1
        if snake_head[0] - 1 < 0 or (snake_head[0] - 1, snake_head[1]) in snake_tail:
            collisionleft = 1
        if snake_head[0] + 1 > frame_size[0] or (snake_head[0] + 1, snake_head[1]) in snake_tail:
            collisionright = 1
    elif dir == 'down':
        if food[1] >= snake_head[1] and food[0] >= snake_head[0]:
            foodq1 = 1
        if food[1] >= snake_head[1] and food[0] <= snake_head[0]:
            foodq2 = 1
        if food[1] <= snake_head[1] and food[0] <= snake_head[0]:
            foodq3 = 1
        if food[1] <= snake_head[1] and food[0] >= snake_head[0]:
            foodq4 = 1
        if snake_head[1] + 1 > frame_size[1] or (snake_head[0], snake_head[1] + 1) in snake_tail:
            collisionfront = 1
        if snake_head[0] + 1 > frame_size[0] or (snake_head[0] + 1, snake_head[1]) in snake_tail:
            collisionleft = 1
        if snake_head[0] - 1 < 0 or (snake_head[0] - 1, snake_head[1]) in snake_tail:
            collisionright = 1
    
    return [foodq1,foodq2,foodq3,foodq4,collisionfront,collisionleft,collisionright]

In [4]:
def calc_reward(found_food, game_over, game_done, move_count):
    reward = 0
    reward -= move_count
    if found_food:
        reward += 20
    if game_over:
        reward -= 0
    elif game_done:
        reward += 0
    return reward

In [ ]:
# "No Brain" Agent
episodes = 10
for episode in range(1, episodes+1):
    env.reset()
    done = False
    reward = 1


    while not done:
        n_state = env.get_state()
        action = random.choice(actions)
        found_food, game_over, game_done, score, move_count = env.play_step(action)
        done = game_over or game_done
        reward += calc_reward(found_food, game_over, game_done, 1)
    print('Episode: {} Reward: {} Score: {} Move_count: {}'.format(episode, reward, score, move_count))

In [8]:
import numpy
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

In [9]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [10]:
model = build_model(states,actions.__len__())
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 7)                 0         
                                                                 
 dense (Dense)               (None, 24)                192       
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 3)                 75        
                                                                 
Total params: 867
Trainable params: 867
Non-trainable params: 0
_________________________________________________________________


In [11]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [16]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions.__len__(), nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [17]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

c:\Users\User Ben\.conda\envs\env39\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


AttributeError: 'Sequential' object has no attribute '_compile_time_distribution_strategy'